In [57]:
import pandas
import torchaudio
import torch
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np
from scipy.fft import rfft, rfftfreq
from scipy.ndimage import gaussian_filter
from scipy.signal import hilbert

import sys
sys.path.append("/home/david/Documentos/Codes/postgresql")
from utils_psql import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:


def wave2frames (s, Nt=512):
    # transform wave into array
    s = np.asarray(s)
    # compute the number of frames
    K = len(s)//Nt
    # Reshape the waveform (ie vector) into a serie of frames (ie 2D matrix)
    timeframes = s[0:K*Nt].reshape(-1,Nt).transpose()
    
    return timeframes


def envelope(s, mode='fast', Nt=32):
    if mode == 'fast' :
        # Envelope : take the max (see M. Towsey) of each frame
        frames = wave2frames(s, Nt)
        env = np.max(abs(frames),0) 
    elif mode =='hilbert' :
        # Compute the hilbert transform of the waveform and take the norm 
        # (magnitude) 
        env = np.abs(hilbert(s))  
    else:
        raise Exception("Invalid mode. Mode should be 'fast' or 'hilbert' ")
        
    return env

In [58]:
import maad

files = list(Path('/media/david/hdd/datasets/San Miguel/').rglob('*.[Ww][Aa][Vv]'))

data = []


for i,f in tqdm(enumerate(files),total=len(files)):
    x, sr = torchaudio.load(f)

    if x.shape[0] == 2:
        x = torch.mean(x, dim=0, keepdim=True)

    x = x.numpy()
    envol1 = envelope(x, mode='hilbert')
    power = np.mean(envol1)
    #power = maad.features.temporal_median(x, mode='hilbert')

    rms = np.sqrt(np.sum(x**2)/x.shape[1])
    envol2 = envelope(x/rms, mode='hilbert')
    power_rms = np.mean(envol2)
    #power_rms = maad.features.temporal_median(x/rms, mode='hilbert')
    print(power, power_rms )



 
    cod_audio = f'{f.parts[-2]}_{f.name[:-4]}'
    
    data.append([cod_audio,power,power_rms])


  0%|          | 0/3045 [00:00<?, ?it/s]

0.0047535636 1.2405053
0.0047297585 1.1746267
0.0048638117 1.2385334
0.0044882237 1.244891
0.0050730626 0.7681712
0.005727602 1.240535
0.0052171727 1.2370051
0.0051741884 1.2414091
0.0051866034 1.2377312
0.004993963 1.1572237
0.0052317064 1.2337099
0.005066955 1.2354019
0.0053910348 1.2435642
0.0055038678 1.2246258
0.005946016 1.2305719
0.005027049 1.2364162
0.005515675 1.2326632
0.0055089267 1.1589652
0.0057023554 1.2277403
0.005787682 1.1575407
0.0051732827 1.2191366
0.005170773 1.2191855
0.005338255 1.212852
0.005253254 1.2135412
0.0050588152 1.238798
0.005132146 1.2239954
0.0051289354 1.2208546
0.005017478 1.2426507
0.0052587064 1.2129225
0.004974472 1.2474655
0.005448789 1.204681
0.0052363914 1.2229809
0.0051503484 1.2344978
0.0050590024 1.2444476
0.005597507 1.2119986
0.0053087273 1.2144817
0.0052708816 1.2032435
0.005371587 1.1909283
0.005698234 1.1661412
0.0055695167 1.1612502
0.0060038175 1.1253034
0.006527616 1.0227149
0.005382946 1.1921446
0.005355576 1.1895311
0.005673535 1

In [60]:
df = pd.DataFrame(columns=['cod_audio','power','power_rms'],data=data)
df['power'] = [float(x) for x in df['power']]
df['power_rms'] = [float(x) for x in df['power_rms']]
df

,cod_audio,power,power_rms
0,G0017_20220724_133000,0.004754,1.240505
1,G0017_20220724_133500,0.004730,1.174627
2,G0017_20220724_134000,0.004864,1.238533
3,G0017_20220724_134500,0.004488,1.244891
4,G0017_20220724_135000,0.005073,0.768171
...,...,...,...
3040,G0004_20220725_060000,0.009635,1.162579
3041,G0004_20220725_060500,0.009704,1.151474
3042,G0004_20220725_061000,0.014082,1.143348
3043,G0004_20220725_061500,0.009939,1.166720


In [55]:
db = ConnectDB('/home/david/Documentos/Codes/postgresql/.env')
conn = db.get_conn()  
loader = DataLoaderDB(conn)
loader.insert_data(df,'power')
db.close()

In [ ]:
float(data[0][1])

0.002532958984375